## About the competition

The challenge is a Natural Language Inference (NLI) competition where given two sentences, there are three ways they could be related: one could entail the other, one could contradict the other, or they could be unrelated.

Your task is to create an NLI model that assigns labels of 0, 1, or 2 (corresponding to entailment, neutral, and contradiction) to pairs of premises and hypotheses. Also, the hypothesis and premise are in multiple languages. 

## Approach

I'll be using the augmented data created by translating training data. I've made the dataset public & you can read about it [here](https://www.kaggle.com/aditya08/contradictory-my-dear-watson-translated-dataset). Essentially, it increases the training dataset by translating training data as explained in [this](https://www.kaggle.com/jpmiller/augmenting-data-with-translations) kernel.

* Augment train & test data
* Make K-Fold train & validation split
* Train XLM-Roberta large model from Huggingface per fold
* Test Time Augmentation (TTA) 
    * Make prediction on test data per fold & average the result
    * Groupby test data by "ID" to average the result of original & translated test data 

## Importing Libraries

In [ ]:
import os
os.environ["WANDB_API_KEY"] = "0"
import numpy as np
import pandas as pd
from pathlib import Path
import gc
import random

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.math import softplus, tanh
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Dense, Input, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow_addons.optimizers import RectifiedAdam

import transformers
from transformers import AutoTokenizer, TFXLMRobertaModel

In [ ]:
sns.set_style("darkgrid")

## Constants

In [ ]:
SEED = 42
EPOCHS = 10
MAX_LEN = 96
NUM_SPLITS = 3
LR = 3e-5
BATCH_SIZE = 16

Setting seed for reproducibility

In [ ]:
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Basic Exploration

In [ ]:
path = Path("../input/contradictory-my-dear-watson-translated-dataset/")

In [ ]:
train = pd.read_csv(path/"train_augmented.csv", index_col=["id"])
test = pd.read_csv(path/"test_augmented.csv", index_col=["id"])

In [ ]:
df = pd.concat([train, test])
df.loc[df["label"]!=-1, "type"] = "train"
df.loc[df["label"]==-1, "type"] = "test"

In [ ]:
plt.figure(figsize=(12, 9))
x = sns.countplot(x="language", hue="type", data=df)
_ = plt.title("Language Distribution")
_ = x.set_xticklabels(x.get_xticklabels(), rotation='45')

* Majority of the training & testing samples are of English language and the rest are in minority.
* Number of samples per language in training data appear to be similar
* Langauges in training and test dataset appear in similar ratios 

In [ ]:
plt.figure(figsize=(6, 4))
_ = sns.countplot(x="label", data=train)
_ = plt.title("Label Distribution")

All the 3 classes are evenly distributed

In [ ]:
del df
gc.collect()

## Setting up TPU

### Detection & Initialization
**Detection** - The TPUClusterResolver instance helps locate the TPUs.

**Initialization** - Initialization process takes place when we switch to TPU accelerator, as we connect to remote clusters and initialize TPUs.

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

### Distribution Strategy

Tensorflow provides apis to distribute training across multiple GPUs, multiple machines or TPUs. Tensorflow provides  multiple distribution strategies as follows -
* MirroredStrategy
* TPUStrategy
* MultiWorkerMirroredStrategy
* CentralStorageStrategy
* ParameterServerStrategy

In Tensorflow 2.2 however, only **MirroredStrategy** & **TPUStrategy** are compatible to be used with Keras API & Custom training loops. You can find more detail in this table [here](https://www.tensorflow.org/guide/distributed_training#types_of_strategies)

**MirroredStrategy** - Supports distributed training on multiple GPUs on one machine. It creates one replica per GPU device.

**TPUStrategy** - TPUStrategy lets us train tensorflow models on TPUs. Similar to *MirroredStrategy*, all the variables are copied to each core. All-reduce algorithms communicate the updates of these variables & aggregates the result from all the devices.

In [ ]:
# instantiate a distribution strategy
strategy = tf.distribute.experimental.TPUStrategy(tpu)

### Scaling batch size
A general idea is to scale the batch size by the number of TPU cores. For Kaggles, TPU v3-8, the core count is 8. So, we essentially increase the batch size 8-fold.

In [ ]:
BATCH_SIZE = BATCH_SIZE*strategy.num_replicas_in_sync

Loads the tokenzier *(SentencePiece tokenizer)* used in `XLM-Roberta`.

In [ ]:
MODEL = 'jplu/tf-xlm-roberta-large'
TOKENIZER = AutoTokenizer.from_pretrained(MODEL)

## Encoding

In [ ]:
def fast_encode(df):
    text = df[['premise', 'hypothesis']].values.tolist()
    encoded = TOKENIZER.batch_encode_plus(
        text,
        pad_to_max_length=True,
        max_length=MAX_LEN
    )
    return np.array(encoded["input_ids"])

To avoid data starvation to TPUs & achieve peak performance, we need an efficient input pipeline that delivers data for the next step before the current step has finished. We can apply the following transformation on training dataset `(tf.data.Datset)`. Refer [here](https://www.tensorflow.org/guide/data_performance) for more details

* `prefetch()`
* `batch()`
* `caching()`

In [ ]:
test_encoded = fast_encode(test)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_encoded)
    .batch(BATCH_SIZE)
)

## Model

*Mish* is a smooth activation function. It calculates the following -

$x * tanh(softplus(x))$

In [ ]:
# softplus - log(exp(x)+1)
def mish(x):
    return x*tanh(softplus(x))
get_custom_objects()["mish"] = Activation(mish)

In [ ]:
def create_model(transformer):
    """
    Fine-Tuning XLM-Roberta by adding a couple of dense layers & dropout.
    Adds a dense layer at the end for 3 labels
    """
    input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32)
    sequence_output = transformer(input_ids)[0]
    cls_token = sequence_output[:, 0, :]
    cls_token = Dropout(0.3)(cls_token)
    cls_token = Dense(32, activation='mish')(cls_token)
    cls_token = Dense(16, activation='mish')(cls_token)
    out = Dense(3, activation='softmax')(cls_token)

    optimizer = RectifiedAdam(lr=LR)
    model = Model(inputs=input_ids, outputs=out)
    model.compile(
        optimizer, 
        loss='sparse_categorical_crossentropy', 
        metrics=['accuracy']
    )
    
    return model

## Training

* Make Stratified 5-Fold split
* Train the model per fold
* Predict & store test predictions per model
* Combine them in the end by dividing it by number of folds & taking the maximum index label

In [ ]:
kfold = StratifiedKFold(n_splits=NUM_SPLITS, shuffle=True, random_state=SEED)

In [ ]:
# learning rate scheduler
def build_lrfn(lr_start=0.00001, lr_max=0.00004, 
               lr_min=0.000001, lr_rampup_epochs=3, 
               lr_sustain_epochs=0, lr_exp_decay=.5):
    lr_max = lr_max * strategy.num_replicas_in_sync
    def lrfn(epoch):
        if epoch < lr_rampup_epochs:
            lr = (lr_max - lr_start) / lr_rampup_epochs * epoch + lr_start
        elif epoch < lr_rampup_epochs + lr_sustain_epochs:
            lr = lr_max
        else:
            lr = (lr_max - lr_min) * lr_exp_decay**(epoch - lr_rampup_epochs - lr_sustain_epochs) + lr_min
        return lr
    return lrfn

In [ ]:
lrfn = build_lrfn()
lrs = LearningRateScheduler(lrfn, verbose=1)

In [ ]:
plt.figure(figsize=(9, 6))
epochs = list(range(EPOCHS))
learning_rates = [lrfn(i) for i in epochs]
plt.plot(epochs, learning_rates)
_ = plt.title("Learning Rate Schedule")
_ = plt.xlabel("# Epcohs")
_ = plt.ylabel("Learning Rate")

In [ ]:
# storing labels only
oof_preds = np.zeros((len(train)))
# storing probabilities per class
test_preds = np.zeros((len(test), 3))

To use K-Fold on TPUs without causing memory issues (refer discussion [here](https://www.kaggle.com/c/flower-classification-with-tpus/discussion/131045)) - 
* Call `tf.tpu.experimental.initialize_tpu_system(tpu)` between each model training to completely reinitialize the TPU
* We can use `drop_remainder=True` while training to increase speed (TPUs run very slowly on final partial batch)

In [ ]:
for fold, (train_index, valid_index) in enumerate(kfold.split(train, train['label'])):
    tf.tpu.experimental.initialize_tpu_system(tpu)

    print("*"*60)
    print("*"+" "*26+f"FOLD {fold+1}"+" "*26+"*")
    print("*"*60, end="\n\n")
    
    X_train = train.iloc[train_index, :].reset_index(drop=True)
    X_valid = train.iloc[valid_index, :].reset_index(drop=True)
    
    y_train = X_train['label'].values
    y_valid = X_valid['label'].values
    
    train_encoded = fast_encode(X_train)
    valid_encoded = fast_encode(X_valid)

    train_dataset = tf.data.Dataset.from_tensor_slices((train_encoded, y_train))
    train_dataset = train_dataset.repeat()
    train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
    train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    valid_dataset = tf.data.Dataset.from_tensor_slices((valid_encoded, y_valid))
    valid_dataset = valid_dataset.batch(BATCH_SIZE)
    valid_dataset = valid_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    num_steps = len(X_train)//BATCH_SIZE
    
    # instantiating the model in the strategy scope creates the model on the TPU
    with strategy.scope():
        transformer_layer = TFXLMRobertaModel.from_pretrained(MODEL)
        model = create_model(transformer_layer)
    
    history = model.fit(
        train_dataset,
        steps_per_epoch=num_steps,
        validation_data=valid_dataset,
        epochs=EPOCHS,
        callbacks=[lrs]
    ) 
    
    # stores validation data prediction at respective indices
    valid_preds = model.predict(valid_dataset)
    oof_preds[valid_index] = valid_preds.argmax(axis=1)
    
    # adds up test prediction per fold
    preds = model.predict(test_dataset)
    test_preds += preds/NUM_SPLITS

In [ ]:
print(f"Accuracy: {accuracy_score(train['label'], oof_preds)}")

## Submission

In [ ]:
print(f"Prediction Shape: {test_preds.shape}")
print(f"Predictions:\n{test_preds[:5]}")

In [ ]:
test["prediction"] = test_preds.tolist()
# groupby "id" original & translated version
submission = test.groupby(by='id')['prediction'].apply(lambda x: [sum(y) for y in zip(*x)]).reset_index()

In [ ]:
# assigning index from sample_submission file
sample_submission = pd.read_csv("../input/contradictory-my-dear-watson/sample_submission.csv")
submission = submission.set_index("id")
submission = submission.reindex(index=sample_submission["id"])
submission = submission.reset_index()

In [ ]:
submission['prediction'] = submission["prediction"].apply(lambda x: np.argmax(x))
submission.to_csv("submission.csv", index=False)

In [ ]:
submission["prediction"].value_counts()

Kindly consider upvoting the kernel, if you find it useful!